# Assignment-1. Mykola Pavlov

## Part 1

We define $\boldsymbol{W}^{(in)} \in \mathbb{R}^{M \times D}$, $\boldsymbol{W}^{(rec)} \in \mathbb{R}^{M \times M}$, $\boldsymbol{x} \in \mathbb{R}^{D \times 1}$, $\boldsymbol{W}^{(out)} \in \mathbb{R}^{P \times M}$, $\boldsymbol{b}^{(1)} \in \mathbb{R}^{M \times 1}$, $\boldsymbol{b}^{(2)} \in \mathbb{R}^{P \times 1}$ (1.1)

Let's rewrite all four expressions in vector form:

$\boldsymbol{a}^{(1)}(k) = \boldsymbol{W}^{(in)} \boldsymbol{x} + \boldsymbol{W}^{(rec)} \boldsymbol{z}^{(1)} (k - 1) + \boldsymbol{b}^{(1)}$

$\boldsymbol{z}^{(1)} = f(\boldsymbol{a}^{(1)})$

$\boldsymbol{a}^{(2)}(k) = \boldsymbol{W}^{(out)} \boldsymbol{z}^{(1)} + \boldsymbol{b}^{(2)}$

$\hat{\boldsymbol{y}} = f(\boldsymbol{a}^{(2)})$

As a result: $\boldsymbol{z}^{(1)} \in \mathbb{R}^{M \times 1}$, $\boldsymbol{a}^{(1)} \in \mathbb{R}^{M \times 1}$, $\boldsymbol{a}^{(2)} \in \mathbb{R}^{P \times 1}$, $\hat{\boldsymbol{y}} \in \mathbb{R}^{P \times 1}$

Now let's assume that we use minibatches of size $B$. Since input is now a matrix $\boldsymbol{X} \in \mathbb{R}^{D \times B}$, how our expressions should be changed?

$\boldsymbol{b}^{(1)}$, $\boldsymbol{b}^{(2)}$ becomes a bias matrices of $\boldsymbol{B}^{(1)} \in \mathbb{R}^{M \times B}$, $\boldsymbol{B}^{(2)} \in \mathbb{R}^{P \times B}$ which consist of repetitions of same vectors $\boldsymbol{b}^{(1)}$, $\boldsymbol{b}^{(2)}$ just B times.

$\boldsymbol{A}^{(1)}(k) = \boldsymbol{W}^{(in)} \boldsymbol{X} + \boldsymbol{W}^{(rec)} \boldsymbol{Z} (k - 1) + \boldsymbol{B}^{(1)}$

$\boldsymbol{Z} = f(\boldsymbol{A}^{(1)})$

$\boldsymbol{A}^{(2)}(k) = \boldsymbol{W}^{(out)} \boldsymbol{Z} + \boldsymbol{B}^{(2)}$

$\hat{\boldsymbol{Y}} = f(\boldsymbol{A}^{(2)})$

As a result: $\boldsymbol{Z} \in \mathbb{R}^{M \times B}$, $\boldsymbol{A}^{(1)} \in \mathbb{R}^{M \times B}$, $\boldsymbol{A}^{(2)} \in \mathbb{R}^{P \times B}$, $\hat{\boldsymbol{Y}} \in \mathbb{R}^{P \times B}$

## Part 2

Let's assume that we want to minimize an error function $E(h) = \mathcal{L}(\hat{y}(h))$ after h time steps using the Simple Neural Network. 

The questions is how the weight matrices $\boldsymbol{W}^{(in)}$, $\boldsymbol{W}^{(rec)}$ and $\boldsymbol{W}^{(out)}$ should be adjusted, so that the objective function achieves the minima.

We can unrolling RNN in time by creating a copy of the model for each time step.
![Backprop in time](bt.png)

As we can see specific $\frac{\partial E(h)}{\partial \boldsymbol{W}^{(out)}}$ only depends on values at the current time step $h$ and additive error from the previous time step $\frac{\partial E(h-1)}{\partial \boldsymbol{W}^{(out)}}$, since we share weights between time steps. $\frac{\partial E(h)}{\partial \boldsymbol{W}^{(out)}} = \sum_{t = 0}^{h} \frac{\partial E(t)}{\partial \boldsymbol{W}^{(out)}} = \sum_{t = 0}^{h} \frac{\partial E(t)}{\partial \hat{y}(t)} \frac{\partial \hat{y}(t)}{\partial a^{(2)}(t)} \frac{\partial a^{(2)}(t)}{\partial \boldsymbol{W}^{(out)}}$ (2.1)

However, $\frac{\partial E(h)}{\partial \boldsymbol{W}^{(rec)}}$ (as well as $\frac{\partial E(h)}{\partial \boldsymbol{W}^{(in)}}$) also depends on $z$ at time step $h - 1$, which it self depends on $\boldsymbol{W}^{(rec)}$ and $z$ at $h - 2$, and so on down to $h = 0$

$\frac{\partial E(h)}{\partial \boldsymbol{W}^{(rec)}} = \frac{\partial E(h)}{\partial \hat{y}(h)} \frac{\partial \hat{y}(h)}{\partial a^{(2)}(h)} \frac{\partial a^{(2)}(h)}{\partial \boldsymbol{W}^{(rec)}} = \sum_{t = 0}^{h} \frac{\partial E(h)}{\partial \hat{y}(h)} \frac{\partial \hat{y}(h)}{\partial a^{(2)}(h)} \frac{\partial a^{(2)}(h)}{\partial z^{(1)}(h)} \frac{\partial z^{(1)}(h)}{\partial z^{(1)}(t)} \frac{\partial z^{(1)}(t)}{\partial a^{(1)}(t)} \frac{\partial a^{(1)}(t)}{\boldsymbol{W}^{(rec)}} $, here $\frac{\partial z^{(1)}(h)}{\partial z^{(1)}(t)}$ could be evaluated as chain rule it self $\prod_{i = t + 1}^{h} \frac{\partial z^{(1)}(i)}{\partial z^{(1)}(i - 1)}$

Note how the gradient signal propagates from layers above (at time step $h$) down to ${a^{(1)}}$ (and also back in time to $t$). For example the error propagation to the time step $t = h - 1$, $\frac{\partial E(h)}{a^{(1)}(h - 1)} = \frac{\partial E(h)}{\partial \hat{y}(h)} \frac{\partial \hat{y}(h)}{\partial a^{(2)}(h)} \frac{\partial a^{(2)}(h)}{\partial z^{(1)}(h)} \frac{\partial z^{(1)}(h)}{\partial z^{(1)}(h-1)} \frac{\partial z^{(1)}(h - 1)}{a^{(1)}(h - 1)}$

Clearly we can generalize this example for any $h$ and previous time step $t$ define a gradient $\frac{\partial E(h)}{a^{(1)}(t)}$ to be:

$\delta^{(h)}_t = \frac{\partial E(h)}{\partial \hat{y}(h)} \frac{\partial \hat{y}(h)}{\partial a^{(2)}(h)} \frac{\partial a^{(2)}(h)}{\partial z^{(1)}(h)} (\prod_{h \geq i \gt t}\frac{\partial z^{(1)}(i)}{\partial z^{(1)}(i-1)}) \frac{\partial z^{(1)}(t)}{a^{(1)}(t)}$ (2.2)

Using this definition of $\delta$ and some initial value for $z$ we can iteratively propagate through time:

$\frac{\partial E(h)}{\partial \boldsymbol{W}^{(rec)}} = \sum_{t = 0}^{h} \delta^{(h)}_t \frac{\partial a^{(1)}(t)}{\boldsymbol{W}^{(rec)}}$ (2.3)

$\frac{\partial E(h)}{\partial \boldsymbol{W}^{(in)}} = \sum_{t = 0}^{h} \delta^{(h)}_t \frac{\partial a^{(1)}(t)}{\boldsymbol{W}^{(in)}}$ (2.4)

## Part 3

Using definitions in Part 1 and expression (2.1) we can derive:

$\nabla_{a^{(2)}(h)} E(h) = \nabla_{\hat{y}(h)}\mathcal{L} * \nabla_{a^{(2)}(h)} f$

$\nabla_{W(out)} E(h) = \sum_{t = 0}^{h} (\nabla_{\hat{y}(t)}\mathcal{L} * \nabla_{a^{(2)}(t)} f) \otimes z^{(1)}$, where $*$ ia elementwise multiplication of two vectors of size $P \times 1$, $\otimes$ is an outer product with a vector of size $M \times 1$ and $\sum$ sums up the resulting matrices of size $M \times P$ elementwise.

Using expressions (2.1) and what we found above we can derive the remeining local derivatives of (2.2):

$\nabla_{z^{(1)}(h)}{a^{(2)}(h)} = \boldsymbol{W}^{(out)}$ of size $P \times M$

$\boldsymbol{J} = \prod_{i = t + 1}^{h} \nabla_{z^{(1)}(i - 1)}{z^{(1)}(i)} = \prod_{i = t + 1}^{h} \boldsymbol{W}^T_{(rec)} diag(\nabla_{z^{(1)}(i - 1)} f )$ is a Jacobian matrix of size $M \times M$

$\delta^{(h)}_t = \boldsymbol{J} [\boldsymbol{W}^T_{(out)} \nabla_{a^{(2)}(h)} E(h)]$ of size $M \times 1$

Using expressions 2.3 and 2.4

$\frac{\partial E(h)}{\partial \boldsymbol{W}^{(rec)}} = \sum_{t = 0}^{h} \delta^{(h)}_t \otimes z(t-1)$ of size $M \times M$

$\frac{\partial E(h)}{\partial \boldsymbol{W}^{(in)}} = \sum_{t = 0}^{h} \delta^{(h)}_t \otimes x(t)$ of size $M \times D$